## Summary of results

- Number of classes: 10, 20, 30, 40
- Basemodel: EfficientNetB0, EfficientNetB1, Xception
- Class weights: true/false
- Number of training data
- Naive labels vs ORES labels
- Affine augmentations of training set
<!-- - Dimension to which image is reduced (now 32x32) -->
<!-- - Unfrozen layers -->

In [2]:
import json
import help_functions as hf
hf.setup_gpu(gpu_nr=0)

2 Physical GPUs, 1 Logical GPU


In [3]:
print('Creating file...')
training_configurations = dict()
    
def create_training_entry(i, 
                          # Data-level
                          augment=False, undersample=False, oversample=False,
                          # Algorithm-level
                          class_weights=False, per_class_thresh=False, loss_function='binary_crossentropy', 
                          # Other methods
                          random_initialization=False, number_trainable_layers=0,
                          hierarchical=False,
                          # Training setup
                          data_folder='', 
                          epochs=20, 
                          basemodel='EfficientNetB2', 
                          nr_classes='all', 
                          batch_size=512, 
                          image_dimension=64,
                          ):
    print(f'Creating {i}th entry in the training_configurations dictionary.')
    training_configurations[i] = dict()
    # Data-level
    training_configurations[i]['augment'] = augment
    training_configurations[i]['oversample'] = oversample
    training_configurations[i]['undersample'] = undersample

    # Algorithm-level
    training_configurations[i]['loss_function'] = loss_function
    training_configurations[i]['class_weights'] = class_weights
    training_configurations[i]['per_class_thresh'] = per_class_thresh

    # Others
    training_configurations[i]['random_initialization'] = random_initialization
    training_configurations[i]['number_trainable_layers'] = number_trainable_layers
    training_configurations[i]['hierarchical'] = hierarchical

    # Training setup
    training_configurations[i]['data_folder'] = data_folder
    training_configurations[i]['batch_size'] = batch_size
    training_configurations[i]['basemodel'] = basemodel
    training_configurations[i]['epochs'] = epochs
    training_configurations[i]['nr_classes'] = nr_classes
    training_configurations[i]['image_dimension'] = image_dimension

    results_folder = f'results_thesis/{i}'
    results_folder += '_augment_' if augment == True else ''
    results_folder += '_oversample_' if oversample == True else ''
    results_folder += '_undersample_' if undersample == True else ''
    results_folder += '_class_weights_' if class_weights == True else ''
    results_folder += '_hierarchical_' if hierarchical == True else ''
    results_folder += '_random_initialization_' if random_initialization == True else ''
    results_folder += f'_bs_{batch_size}'
    training_configurations[i]['results_folder'] = results_folder


Creating file...


In [4]:
# create_training_entry(i=0, random_initialization=False, number_trainable_layers=339)
# create_training_entry(i=1, random_initialization=True, number_trainable_layers=339)

# create_training_entry(i=10, 
#                       random_initialization=False, 
#                       number_trainable_layers=0, 
#                       batch_size=32, 
#                       undersample=True)

# create_training_entry(i=11, 
#                       random_initialization=False, 
#                       number_trainable_layers=0, 
#                       batch_size=32, 
#                       oversample=True)

# create_training_entry(i=20, 
#                       data_folder='data/split_hierarchical_data_221115',
#                       nr_classes='all')

FLAT_LABELS_PATH = 'data/split_flat_data_221130'
HIERARCHICAL_LABELS_PATH = 'data/split_hierarchical_data_221129'

# ==================== DATA-LEVEL TECHNIQUES ====================
# 0. Standard transfer-learning setup (BCE loss, 0.5 thresholds, 
#    equal class-weights, no resampling, no augmentation, weights 
#    pre-trained on ImageNet, fine-tuning only the extra two layers, 
#    etc)
create_training_entry(i=100, 
                      # Data-level
                      augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                      class_weights=False, loss_function='binary_crossentropy', per_class_thresh=False,
                      # Other methods
                      random_initialization=False, number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)

# 1. Undersampling
create_training_entry(i=101, 
                      # Data-level
                      undersample=True,       augment=False, oversample=False,
                      # Algorithm-level
                                              class_weights=False, per_class_thresh=False,
                      # Other methods
                                              random_initialization=False, number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2', 
                      nr_classes='all', 
                      batch_size=512, 
                      loss_function='binary_crossentropy', 
                      image_dimension=64)

# 2. Oversampling
create_training_entry(i=102, 
                      # Data-level -----------------------------------------------------------------
                      oversample=True,        augment=False, undersample=False,
                      # Algorithm-level ------------------------------------------------------------
                                              class_weights=False, per_class_thresh=False,
                      # Other methods --------------------------------------------------------------
                                              random_initialization=False, number_trainable_layers=0,
                      # Training setup -------------------------------------------------------------
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2', 
                      nr_classes='all', 
                      batch_size=512, 
                      loss_function='binary_crossentropy', 
                      image_dimension=64)

# 3. Augmentation
create_training_entry(i=103, 
                      # Data-level -----------------------------------------------------------------
                      augment=True,           undersample=False, oversample=False,        
                      # Algorithm-level ------------------------------------------------------------
                                              class_weights=False, per_class_thresh=False,
                      # Other methods --------------------------------------------------------------
                                              random_initialization=False, number_trainable_layers=0,
                      # Training setup -------------------------------------------------------------
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2', 
                      nr_classes='all', 
                      batch_size=512, 
                      loss_function='binary_crossentropy', 
                      image_dimension=64)

# ==================== ALGORITHM-LEVEL TECHNIQUES ====================
# 4. Class-weights
create_training_entry(i=104, 
                      # Data-level
                                                augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                      class_weights=True,       loss_function='binary_crossentropy', per_class_thresh=False,
                      # Other methods
                                                random_initialization=False, number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)

# 5. Per-class thresholds
create_training_entry(i=105, 
                      # Data-level
                                                   augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                      per_class_thresh=True,       class_weights=False, loss_function='binary_crossentropy', 
                      # Other methods
                                                   random_initialization=False, number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)

# 6. Loss function
create_training_entry(i=106, 
                      # Data-level
                                                   augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                      loss_function='focal_loss',  per_class_thresh=False, class_weights=False,  
                      # Other methods
                                                   random_initialization=False, number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)

# ==================== OTHER EXPERIMENTS ====================
# 7. Randomized weights
create_training_entry(i=107, 
                      # Data-level
                                                   augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                                                   loss_function='binary_crossentropy', per_class_thresh=False, class_weights=False,  
                      # Other methods
                      random_initialization=True,  number_trainable_layers=0,
                      # Training setup
                      data_folder=FLAT_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)

# 8. Fine-tuning layers # use the data we already used
# create_training_entry(i=108, 
#                       # Data-level
#                                                    augment=False, undersample=False, oversample=False,
#                       # Algorithm-level
#                                                    loss_function='binary_crossentropy', per_class_thresh=False, class_weights=False,  
#                       # Other methods
#                       number_trainable_layers=339, random_initialization=False,  
#                       # Training setup
#                       data_folder='data/split_dataframes_heuristic_labels_20221006', 
#                       epochs=20, 
#                       basemodel='EfficientNetB2',
#                       nr_classes='all', 
#                       batch_size=512, 
#                       image_dimension=64)


# 9. Hierarchical
create_training_entry(i=109, 
                      # Data-level
                                                   augment=False, undersample=False, oversample=False,
                      # Algorithm-level
                                                   loss_function='binary_crossentropy', per_class_thresh=False, class_weights=False,  
                      # Other methods
                                                   random_initialization=False,  number_trainable_layers=0,
                      # Training setup
                      data_folder=HIERARCHICAL_LABELS_PATH, 
                      epochs=20, 
                      basemodel='EfficientNetB2',
                      nr_classes='all', 
                      batch_size=512, 
                      image_dimension=64)



with open('training_configurations.json', 'w') as fp:
    json.dump(training_configurations, fp)

Creating 100th entry in the training_configurations dictionary.
Creating 101th entry in the training_configurations dictionary.
Creating 102th entry in the training_configurations dictionary.
Creating 103th entry in the training_configurations dictionary.
Creating 104th entry in the training_configurations dictionary.
Creating 105th entry in the training_configurations dictionary.
Creating 106th entry in the training_configurations dictionary.
Creating 107th entry in the training_configurations dictionary.
Creating 109th entry in the training_configurations dictionary.
